# 目的：根据高血压药物的ATC代码，抽取出只有高血压药物的prescriptions.csv

In [ ]:
import pandas as pd
import numpy as np
import json
from pandas import Series,DataFrame
import warnings
warnings.filterwarnings("ignore")

# 原始prescriptions.csv

In [ ]:
med_pd = pd.read_csv(r'E:\06.数据集备份\mimic-iv-高血压\prescriptions.csv', dtype={'ndc':'category'})
print('行数：', len(med_pd))
med_pd.head(5)

# NDC代码转换成xrnorm，xrnorm再转换成atc4格式

In [ ]:
ndc_rxnorm_file = './ndc2rxnorm_mapping.txt'  # 药物代码映射文件
with open(ndc_rxnorm_file, 'r') as f:
    ndc2rxnorm = eval(f.read())
# ndc2rxnorm是个字典，key为ndc，value为rxcui
med_pd['rxcui'] = med_pd['ndc'].map(ndc2rxnorm)
print('行数：', len(med_pd))
med_pd.head(5)

In [ ]:
# med_pd.replace("nan",np.nan,inplace=True)
med_pd = med_pd.dropna(subset=['rxcui'], how='any') # 删除空行
print('行数：', len(med_pd))
med_pd.head(5)

In [ ]:
rxnorm2atc_file = './rxnorm2atc4.csv'
rxnorm2atc = pd.read_csv(rxnorm2atc_file)  # 读取rxnorm2atc文件
rxnorm2atc = rxnorm2atc.drop(columns=['YEAR', 'MONTH', 'NDC'])  # 只留rxcui、atc4两列
rxnorm2atc.drop_duplicates(subset=['RXCUI'], inplace=True)  # 根据rxcui值去重
rxnorm2atc = rxnorm2atc.rename(columns={'RXCUI': 'rxcui', 'ATC4': 'atc'})
rxnorm2atc

In [ ]:
rxnorm2atc['rxcui'] = rxnorm2atc['rxcui'].astype('str')  #改变数据类型为string
# med = med_pd.reset_index(drop=True)  # 重置索引
med_pd = med_pd.merge(rxnorm2atc, on=['rxcui'])
print('行数：', len(med_pd))
med_pd.head(5)

# 抽取高血压药物

In [ ]:
med_pd['atc'] = med_pd['atc'].map(lambda x: x[:3])  # 取atc前三个值
print('行数：', len(med_pd))
med_pd.head(5)

In [ ]:
# 根据高血压药物ATC值抽药物
med_pd = med_pd[(med_pd['atc']=='C02') | (med_pd['atc']=='C03') | (med_pd['atc']=='C07') | (med_pd['atc']=='C08') | (med_pd['atc']=='C09')]
print('行数：', len(med_pd))
med_pd.head(5)

In [ ]:
med_pd = med_pd.drop(columns = ['rxcui', 'atc'])
med_pd.to_csv(r'E:\06.数据集备份\mimic-iv-高血压\prescriptions.csv', index=False)
print('行数：', len(med_pd))
med_pd.head(5)